## A Chatbot using GPT and a Database
This allows multiple chatbot types (e.g. a health coach and a learning assistant) to be created. Multiple chatbot instances can be created per chatbot type (e.g. for User X and User Y). Both, type and instance are stored and referenced with an ID (e.g. with a UUID) in the database.

This can support the deployment of chatbots in a web backend (state-less). For example, the UUIDs of the type and instance can be read as URL parameters from a URL that users have received from you.

A chatbot is created with the following arguments.
- database_file: File of SQLite (in Folder data/)
- type_id: Reference to chatbot type
- instance_id: Reference to chatbot instance (typically one per user - however, may also be shared by multiple users)
- type_role: Role of chatbot type (will be turned into a first prompt with role:system)
- instance_context: Context of chatbot instance (will be turned into a second prompt with role:system)
- instance_starter: Will be used to generate an initial message to the user (will be turned into a final prompt with role:system)

The following functions are meant to be used from an application (e.g. from controllers of a REST API).
- conversation_retrieve(with_system=False): Retrieve the previous conversation history (default: without prompts with role:system)
- start(): Returns an initial message to the user (Resulting from instance_starter prompt)
- respond(user_says): Returns an assistance response to user_says
- info_retrieve(): Returns the chatbot name, type role and instance context
- reset(): Resets the conversation so far

#### OpenAI API Key and Model

Create file chatbot/openai.py with the following content\
    OPENAI_KEY = "your OpenAI API key"\
    OPENAI_MODEL = "gpt-3.5-turbo-16k"\
(You may rename the file chatbot/openai_template.py to openai.py and set the keys there)

#### Google Colab Only: START

You need a Google and Github account. Be prepared to authenticate yourself with both of these accounts.

#### Google Colab Only: END

In [1]:
from chatbot.chatbot import Chatbot

In [ ]:
role = """
Du bist ein Schriftsteller, der eine personalisierte Geschichte für einen Benutzer erstellen möchte. Dein Ziel ist es, in einem Gespräch mit dem Benutzer Antworten auf bestimmte Fragen zu erhalten, die dir helfen, die Geschichte auf seine Vorlieben abzustimmen. Stelle nicht alle Fragen auf einmal, sondern führe ein natürliches Gespräch, bei dem du schrittweise die benötigten Informationen erhältst. Die Person sollte sich nicht wie beim Ausfüllen eines Formulars fühlen. Erzeuge immer kurze, ansprechende Nachrichten an den Benutzer, um das Gespräch lebendig und interessant zu gestalten. Hier sind die Fragen, die du im Laufe des Gesprächs stellen sollst:

1. Was sind deine Lieblingsbücher oder -autoren und was magst du besonders an ihnen?
2. Welche Art von Genres gefällt dir am meisten, wie z.B. Fantasy, Science-Fiction, Krimis oder Abenteuer?
3. Welche Arten von Charakteren findest du am interessantesten oder inspirierendsten?
4. Gibt es bestimmte Themen oder Botschaften, die dich in Geschichten besonders ansprechen?
5. Bevorzugst du Geschichten, die in der realen Welt, in einer fantastischen Welt, in der Zukunft oder in der Vergangenheit spielen?
6. Welche Stimmung magst du in Geschichten am liebsten – spannend, lustig, gruselig, nachdenklich oder romantisch?
7. Welche Filme oder Serien magst du und warum? Dies kann mir helfen, deinen Geschmack besser zu verstehen.
8. Erzähle mir von deinen Hobbies oder Interessen. Vielleicht kann ich diese in die Geschichte einbauen.
9. Gibt es spezielle Elemente, die du in einer Geschichte gerne sehen würdest, wie z.B. magische Fähigkeiten, Zeitreisen oder besondere Rätsel?
10. Gibt es Dinge in Geschichten, die du nicht magst oder lieber vermeiden möchtest?

Deine Aufgabe ist es, das Gespräch so zu führen, dass du am Ende alle notwendigen Informationen hast, um eine ansprechende und individuell zugeschnittene Geschichte zu schreiben.
"""
instance = """
Du führst nun ein Gespräch mit einer Benutzerin. Diese heisst Zoe und ist 13 Jahre alt. Führe das Gespräch auf eine motivierende, unterstützende und wohlwollende Art. Ermutige sie zur Beteiligung an das Gespräch und vermeide lange Antworten an Zoe.

"""
starter = """

"""

#### Create a chatbot "Coach" for user X

In the following, we use the default type_name, type_role, instance_context, and instance_starter defined in the Chatbot class. Provide your own prompts instead.

In [ ]:
bot = Chatbot(
    database_file="database/chatbot.db", 
    type_id="5dbc5649-834c-44c4-9109-0567e183a2ec",
    user_id="43517942-1c16-4d41-86ff-5636d202b81d",
    type_name=Chatbot.default_type_name,
    type_role=Chatbot.default_type_role,
    instance_context=Chatbot.default_instance_context,
    instance_starter=Chatbot.default_instance_starter
)

In [ ]:
print(bot.start())

#### Creating multiple instances of chatbot "Coach"
In the following, we assume the existence of the bot type created in the cells above. We show example code that will generate N bot instances of that type. Each instance has it's own prompts (instance context and starter) that will be appended to the type prompts. Most importantly, each instance has its own chat history.

In [ ]:
import uuid
import time

In [ ]:
# Amount of instances to be created
number_of_instances = 1

# Change the following to a list of hardcoded instance IDs if you want to use existing users.
user_ids = [str(uuid.uuid4()) for _ in range(number_of_instances)]

c  = 0 # counter for successful requests, don't change
error_c = 0 # counter for failed requests, don't change
for user_id in user_ids:
    bot = Chatbot(
        database_file="database/chatbot.db", 
        type_id="5dbc5649-834c-44c4-9109-0567e183a2ec",
        user_id=user_id,
        instance_context=Chatbot.default_instance_context,
        instance_starter=Chatbot.default_instance_starter
    )
    try:
        # each bot should have a first message to the user
        print(bot.start())
    except:
        error_c += 1
        continue
    c+=1
    time.sleep(15) #openai seems to produce more errors if we send the requests too fast.
    
print("successful: {}, failed: {}".format(c, error_c))


#### Obtain URLs of all instances of a type
We need one instance of that type and can then use the type_instances() function to retrieve all of instance ids. Using these instance ids we can then create URLs such as for pythonanywhere environment.

In [ ]:
pythonanywhere_username = "<ENTER YOUR PYTHONANYWHERE USERNAME HERE>"
type_id = "5dbc5649-834c-44c4-9109-0567e183a2ec"
bot = Chatbot(
    database_file="database/chatbot.db", 
    type_id=type_id,
    user_id=user_ids[0]
)

for user_id in bot.type_instances():
    print("https://{}.pythonanywhere.com/{}/{}/chat".format(pythonanywhere_username, type_id, user_id))